In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
session = tf.compat.v1.InteractiveSession(config=config)

In [3]:
def AN(X, y, cnt):
    # train, test split
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)
   
    #print(len(y))
    #print(len(y_train))
    #print(X_train[0].shape)
    AlexNet = keras.models.Sequential([
            keras.layers.Conv2D(96, (6, 6), strides=4, activation='relu', input_shape=X_train[0].shape),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),
        
            keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Flatten(),
            keras.layers.Dense(1024, activation='relu'),
            keras.layers.Dense(1024, activation='relu'),

            keras.layers.Dense(4, activation='softmax')
    ])
    #AlexNet.summary()
    AlexNet.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(learning_rate=0.01), 
            metrics=[keras.metrics.sparse_categorical_accuracy])
    
     
    AlexNet.fit(X_train, y_train, epochs=10, batch_size=16,
                    validation_data=(X_validation, y_validation))
    '''
    import matplotlib.pyplot as plt

    #print(AlexNet_history.history)

    plt.plot(AlexNet_history.history['val_sparse_categorical_accuracy'])
    plt.legend(['val_acc'], loc= 'upper left')
    plt.show()
    '''
    evaluation = []
    evaluation.append(AlexNet.evaluate(X_train, y_train, batch_size=16))
    #print(f"cnt = {cnt}, {evaluation}")
    evaluation.append(AlexNet.evaluate(X_test, y_test, batch_size=16))
    #print(f"cnt = {cnt}, {evaluation}")
    return evaluation

In [4]:
import os, re, glob
import cv2

image_size = 227

groups_folder_path = '../Data/resultVecsFigs/AAFT_0/'

X_s = []

y = []
# auto_corr       hadamard        midAng     striping_color      striping_midAng_color
for label in ['Bi', 'Tri', 'Hammer', 'Rvcurl']:

    for feature in ['hadamard']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            #print(f)
            for filename in f:
                #print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])
                
                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                elif label == 'Tri':
                    y.append(1)
                elif label == 'Hammer':
                    y.append(2)
                else:
                    y.append(3)

print(X_s[0].shape)
#print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

an = []

for i in range(10):
    an.append(AN(X, y, i))

(227, 227, 3)
240
Epoch 1/10
11/11 [==============================] - 8s 386ms/step - loss: 0.6687 - sparse_categorical_accuracy: 0.7685 - val_loss: 1.2960 - val_sparse_categorical_accuracy: 0.8500
Epoch 2/10
11/11 [==============================] - 0s 19ms/step - loss: 0.0723 - sparse_categorical_accuracy: 0.9957 - val_loss: 1.2171 - val_sparse_categorical_accuracy: 0.5000
Epoch 3/10
11/11 [==============================] - 0s 18ms/step - loss: 0.0363 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.1422 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/10
11/11 [==============================] - 0s 18ms/step - loss: 0.0216 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0801 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0120 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0317 - val_sparse_categorical_accuracy: 0.5000
Epoch 6/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0122 - spar

Epoch 2/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0511 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.1932 - val_sparse_categorical_accuracy: 0.8000
Epoch 3/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0222 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.1158 - val_sparse_categorical_accuracy: 0.6000
Epoch 4/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0246 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0439 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/10
11/11 [==============================] - 0s 18ms/step - loss: 0.0200 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9848 - val_sparse_categorical_accuracy: 0.5000
Epoch 6/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0110 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9368 - val_sparse_categorical_accuracy: 0.5000
Epoch 7/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0069 - sparse_categorical_accu

11/11 [==============================] - 0s 17ms/step - loss: 0.0489 - sparse_categorical_accuracy: 0.9990 - val_loss: 1.1409 - val_sparse_categorical_accuracy: 0.7000
Epoch 4/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0252 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0758 - val_sparse_categorical_accuracy: 0.6000
Epoch 5/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0161 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0153 - val_sparse_categorical_accuracy: 0.8000
Epoch 6/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0096 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9623 - val_sparse_categorical_accuracy: 0.9500
Epoch 7/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0098 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9135 - val_sparse_categorical_accuracy: 1.0000
Epoch 8/10
11/11 [==============================] - 0s 17ms/step - loss: 0.0138 - sparse_categorical_accuracy: 1.000

In [5]:
for data in an:
        print(data[0][0])

0.8980162739753723
0.9487863779067993
0.8760440945625305
0.8387609124183655
0.8212676048278809
0.8817518353462219
0.7313274145126343
0.8759732842445374
0.7761838436126709
0.8451616764068604


In [6]:
for data in an:
        print(data[1][1])

0.5
0.5
0.5
0.5
0.5
0.9166666865348816
0.8333333134651184
0.5
0.9791666865348816
0.5
